# Inside the Decentralised Casino

In [2]:
# The first step is to load in gamba so we can use its methods
import sys
import pandas as pd
pd.options.display.float_format = '{:,.3f}'.format
sys.path.append('/home/ojs/Dropbox/phd/code/python/gamba_release')
import gamba as gb, importlib
importlib.reload(gb)

gamba loaded, see https://gamba.dev/citation
gamba loaded, see https://gamba.dev/citation


<module 'gamba' from '/home/ojs/Dropbox/phd/code/python/gamba_release/gamba/__init__.py'>

In [3]:
raw_data = gb.read_csv('/home/ojs/Desktop/raw_matched_data.csv', ['bet_time','payout_time','duration'])
gb.summarise_app(raw_data)

users: 24234
games: 5
num bets: 2232741
bet value: 2723287.792136176
payout value: 2688008.0368767725
start: 2018-08-04 04:27:21 6084746
end: 2020-03-09 19:32:55 9639151



In [4]:
df = raw_data

providers = ['dice2win','fck','etheroll']
games = ['coinflip','onedice','twodice','roll']
all_games = []
game_players = []
for provider in providers:
    for game in games:
        game_provider_combo = df[(df['provider'] == provider) & (df['game_type'] == game)]
        if len(game_provider_combo) == 0:
            continue
        all_games.append(game_provider_combo)
        game_players.append(set(game_provider_combo['player_id'].values))
        print('n=', len(set(game_provider_combo['player_id'])), 'b=', len(game_provider_combo))
    
for provider in providers:
    print(provider)
    gb.summarise_app(df[(df['provider'] == provider) & (df['game_type'].isin(games))])

n= 5742 b= 702502
n= 2979 b= 240728
n= 1633 b= 167759
n= 3014 b= 416134
n= 13877 b= 137011
n= 567 b= 48331
n= 293 b= 27163
n= 396 b= 36316
n= 3086 b= 362324
dice2win
users: 7868
games: 4
num bets: 1527123
bet value: 1267239.950539507
payout value: 1245815.2785478367
start: 2018-09-07 08:17:21 6287216
end: 2020-03-09 19:32:55 9639151

fck
users: 14466
games: 4
num bets: 248821
bet value: 465195.8532056361
payout value: 462136.7118950764
start: 2018-12-10 06:05:13 6859200
end: 2019-07-02 08:49:06 8071084

etheroll
users: 3086
games: 1
num bets: 362324
bet value: 420942.44164378115
payout value: 419067.602043743
start: 2018-08-04 04:27:21 6084746
end: 2020-03-09 17:35:39 9638617



In [4]:
gb.calculate_labrie_measures('/home/ojs/Desktop/individuals/', filename='dgapp_measures.csv', daily=False)

loading in player CSVs...


  0%|          | 8/24234 [00:00<05:03, 79.73it/s]

calculating LaBrie measures for 24234 players, this may take some time...


100%|██████████| 24234/24234 [09:46<00:00, 41.29it/s]


In [6]:
measures = gb.read_csv('dgapp_measures.csv')

game_measures = []
for players in game_players:
    game_measures.append(measures[measures['player_id'].isin(players)])

labels = ['d2w_f','d2w_d','d2w_2d','d2w_r','fck_f','fck_d','fck_2d','fck_r','eroll']
matrix = gb.calculate_walker_matrix(game_measures, labels, measure='duration')
display(matrix)

,d2w_f,d2w_d,d2w_2d,d2w_r,fck_f,fck_d,fck_2d,fck_r,eroll
d2w_f,-,,,,,,,,
d2w_d,0.160,-,,,,,,,
d2w_2d,0.24**,0.100,-,,,,,,
d2w_r,0.170,0.030,0.080,-,,,,,
fck_f,0.55**,0.39**,0.46**,0.39**,-,,,,
fck_d,0.22**,0.08**,0.050,0.050,0.43**,-,,,
fck_2d,0.4**,0.25**,0.16**,0.23**,0.59**,0.2**,-,,
fck_r,0.34**,0.19**,0.1**,0.17**,0.54**,0.13**,0.070,-,
eroll,0.090,0.120,0.19**,0.110,0.49**,0.14**,0.34**,0.27**,-


In [7]:
# human players are non-fck_flip players
human_df = measures[~measures['player_id'].isin(game_players[4])]
print(len(human_df), human_df['num_bets'].sum())

10357 1743478


In [9]:
table = gb.add_tables(gb.descriptive_table(human_df, extended=True), gb.ks_test(human_df))
spearman_coefficient_table = gb.calculate_labrie_coefficients(human_df)
top5, other95 = gb.split_labrie_measures(human_df)
print(len(top5), len(other95))
comparison_table = gb.add_tables(gb.descriptive_table(top5, extended=True), gb.descriptive_table(other95, extended=True))

518 9839


In [8]:
display(table, spearman_coefficient_table, comparison_table)

,mean,σ,median,iqr,K-S Score,p
duration,29.695,80.645,1.000,10.000,0.841,0.000
frequency,76.382,36.450,100.000,50.000,0.966,0.000
num_bets,168.338,991.976,11.000,62.000,0.841,0.000
average_bets_per_day,23.126,47.872,6.000,21.000,0.841,0.000
average_bet_size,1.152,11.811,0.113,0.353,0.504,0.000
total_wagered,213.770,"2,451.852",1.400,16.590,0.504,0.000
net_loss,2.911,49.864,0.041,0.711,0.213,0.000
percent_loss,10.913,112.137,5.327,51.991,0.548,0.000


,Duration,Frequency,Number of bets,Average Bets per Day,Average Bet Size,Total wagered,Net loss,Percent loss
Duration,-,,,,,,,
Frequency,-0.89**,-,,,,,,
Number of bets,0.63**,-0.45**,-,,,,,
Average Bets per Day,0.35**,-0.19**,0.93**,-,,,,
Average Bet Size,0.16**,-0.1**,0.26**,0.24**,-,,,
Total wagered,0.53**,-0.37**,0.84**,0.78**,0.72**,-,,
Net loss,0.12**,-0.1**,0.15**,0.14**,0.15**,0.2**,-,
Percent loss,-0.1**,0.06**,-0.15**,-0.12**,-0.07**,-0.14**,0.67**,-


,mean,σ,median,iqr,mean,σ,median,iqr
duration,95.560,138.375,35.000,120.000,26.227,74.829,1.000,7.000
frequency,55.033,35.143,50.000,78.349,77.506,36.172,100.000,50.000
num_bets,"1,787.716","3,951.361",644.000,"1,660.000",83.082,264.546,9.000,47.000
average_bets_per_day,90.584,95.034,68.022,77.057,19.575,41.055,5.000,17.690
average_bet_size,13.767,50.531,1.841,5.613,0.488,1.960,0.102,0.279
total_wagered,"3,874.377","10,306.803",986.389,"1,759.015",21.048,53.574,1.100,10.890
net_loss,38.077,216.264,10.302,102.597,1.059,9.535,0.039,0.600
percent_loss,0.947,9.048,0.903,7.598,11.438,115.009,6.556,57.556


In [11]:
gb.ks_test(other95)

,K-S Score,p
duration,0.841,0.000
frequency,0.965,0.000
num_bets,0.841,0.000
average_bets_per_day,0.841,0.000
average_bet_size,0.504,0.000
total_wagered,0.504,0.000
net_loss,0.222,0.000
percent_loss,0.559,0.000
